In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

In [2]:
years = [2015, 2016, 2017, 2018, 2019]
ds_15 = pd.read_csv('data/2015.csv').drop(['Standard Error'],axis=1)
ds_15.rename(columns={"Happiness Rank": "Rank","Happiness Score":"Score","Economy (GDP per Capita)":"GDP","Family":"Social support","Health (Life Expectancy)":"Healthy","Trust (Government Corruption)":"Trust"},inplace=True)  
ds_15.insert(0, "Year", 2015)
ds_15['Rounded Score'] = ds_15['Score'].round(0)

ds_16 = pd.read_csv('data/2016.csv').drop(['Lower Confidence Interval','Upper Confidence Interval'],axis=1)
ds_16.rename(columns={"Happiness Rank": "Rank","Happiness Score":"Score","Economy (GDP per Capita)":"GDP","Family":"Social support","Health (Life Expectancy)":"Healthy","Trust (Government Corruption)":"Trust"},inplace=True) 
ds_16.insert(0, "Year", 2016)
ds_16['Rounded Score'] = ds_16['Score'].round(0)

ds_17 = pd.read_csv('data/2017.csv').drop(['Whisker.high','Whisker.low'],axis=1)
ds_17.rename(columns={"Happiness.Rank": "Rank","Happiness.Score":"Score","Economy..GDP.per.Capita.":"GDP","Family":"Social support","Health..Life.Expectancy.":"Healthy","Trust..Government.Corruption.":"Trust","Dystopia.Residual":"Dystopia Residual"},inplace=True) 
ds_17.insert(0, "Year", 2017)
ds_17 = ds_17.join(ds_15.set_index('Country')['Region'], on='Country')
ds_17['Rounded Score'] = ds_17['Score'].round(0)

ds_18 = pd.read_csv('data/2018.csv')
ds_18.rename(columns={"Country or region": "Country", "Overall rank": "Rank","Score":"Score","GDP per capita":"GDP","Healthy life expectancy":"Healthy","Freedom to make life choices":"Freedom","Perceptions of corruption":"Trust"},inplace=True)
ds_18.insert(0, "Year", 2018)
ds_18 = ds_18.join(ds_15.set_index('Country')['Region'], on='Country')
ds_18['Rounded Score'] = ds_18['Score'].round(0)

ds_19 = pd.read_csv('data/2019.csv')
ds_19.rename(columns={"Country or region": "Country", "Overall rank": "Rank","GDP per capita":"GDP","Healthy life expectancy":"Healthy","Freedom to make life choices":"Freedom","Perceptions of corruption":"Trust"},inplace=True)
ds_19.insert(0, "Year", 2019)
ds_19 = ds_19.join(ds_15.set_index('Country')['Region'], on='Country')
ds_19['Rounded Score'] = ds_19['Score'].round(0)

df = pd.concat([ds_15, ds_16, ds_17, ds_18, ds_19], sort=True, ignore_index=True)

df

,Country,Dystopia Residual,Freedom,GDP,Generosity,Healthy,Rank,Region,Rounded Score,Score,Social support,Trust,Year
0,Switzerland,2.51738,0.66557,1.39651,0.29678,0.94143,1,Western Europe,8.0,7.587,1.34951,0.41978,2015
1,Iceland,2.70201,0.62877,1.30232,0.43630,0.94784,2,Western Europe,8.0,7.561,1.40223,0.14145,2015
2,Denmark,2.49204,0.64938,1.32548,0.34139,0.87464,3,Western Europe,8.0,7.527,1.36058,0.48357,2015
3,Norway,2.46531,0.66973,1.45900,0.34699,0.88521,4,Western Europe,8.0,7.522,1.33095,0.36503,2015
4,Canada,2.45176,0.63297,1.32629,0.45811,0.90563,5,North America,7.0,7.427,1.32261,0.32957,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,Rwanda,NaN,0.55500,0.35900,0.21700,0.61400,152,Sub-Saharan Africa,3.0,3.334,0.71100,0.41100,2019
778,Tanzania,NaN,0.41700,0.47600,0.27600,0.49900,153,Sub-Saharan Africa,3.0,3.231,0.88500,0.14700,2019
779,Afghanistan,NaN,0.00000,0.35000,0.15800,0.36100,154,Southern Asia,3.0,3.203,0.51700,0.02500,2019
780,Central African Republic,NaN,0.22500,0.02600,0.23500,0.10500,155,Sub-Saharan Africa,3.0,3.083,0.00000,0.03500,2019


In [3]:
""" Adding external data. """
edf = pd.read_csv('data/wdidata.csv')

In [ ]:
df['Mujeres trab.agricultura'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        mujeres_campo = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'SL.AGR.EMPL.FE.ZS')][str(year)]
        if mujeres_campo.values.size > 0:
            df.at[index, 'Mujeres trab.agricultura'] = mujeres_campo.values[0]

df['GDP por trabajador'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        gdp_employed = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'SL.GDP.PCAP.EM.KD')][str(year)]
        if gdp_employed.values.size > 0:
            df.at[index, 'GDP por trabajador'] = gdp_employed.values[0]

df['Participación lab.1524'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        part_employ = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'SL.TLF.ACTI.1524.FE.ZS')][str(year)]
        if part_employ.values.size > 0:
            df.at[index, 'Participación lab.1524'] = part_employ.values[0]

df['Mujeres trab.servicios'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        mujeres_servicios = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'SL.SRV.EMPL.FE.ZS')][str(year)]
        if mujeres_servicios.values.size > 0:
            df.at[index, 'Mujeres trab.servicios'] = mujeres_servicios.values[0]

df['Mujeres desempleadas'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        mujeres_desempleadas = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'SL.UEM.TOTL.FE.ZS')][str(year)]
        if mujeres_desempleadas.values.size > 0:
            df.at[index, 'Mujeres desempleadas'] = mujeres_desempleadas.values[0]

df['Hombres mas 80'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        hombres_mayores_80 = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'SP.POP.80UP.MA.5Y')][str(year)]
        if hombres_mayores_80.values.size > 0:
            df.at[index, 'Hombres mas 80'] = hombres_mayores_80.values[0]

df['Trasporte aereo'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        transp_aereo = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'IS.AIR.DPRT')][str(year)]
        if transp_aereo.values.size > 0:
            df.at[index, 'Trasporte aereo'] = transp_aereo.values[0]

df['Mujeres desempleadas.1524'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        mujeres_des1524 = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'SL.UEM.1524.FE.ZS')][str(year)]
        if mujeres_des1524.values.size > 0:
            df.at[index, 'Mujeres desempleadas.1524'] = mujeres_des1524.values[0]

df['Ratio dif empleo genero'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        ratio_dif_empleo = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'SL.TLF.CACT.FM.ZS')][str(year)]
        if ratio_dif_empleo.values.size > 0:
            df.at[index, 'Ratio dif empleo genero'] = ratio_dif_empleo.values[0]

df['Refugiados'] = np.nan
df.head()

for index, row in df.iterrows():
    country_name = row['Country']
    year = row['Year']
    if year in years:
        refugiados_por_pais = edf.loc[(edf['Country Name'] == country_name) & (edf['Indicator Code'] == 'SM.POP.REFG.OR')][str(year)]
        if refugiados_por_pais.values.size > 0:
            df.at[index, 'Refugiados'] = refugiados_por_pais.values[0]

df